In [1]:
import pandas as pd
import psycopg2
import pandas.io.sql as psql
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from pymongo import MongoClient
import requests
import pprint
import time

%matplotlib inline

In [2]:
!ls ../Mahmud_ICWSM_2012/

profiletweets.csv  README.txt  topcitiesusersandtweets.csv


In [3]:
users_and_tweets = pd.read_csv("../Mahmud_ICWSM_2012/topcitiesusersandtweets.csv")

In [4]:
users_and_tweets.head()

,City,State,SW LAT,SW LONG,NE LAT,NE LONG,USER ID,TWEET ID
0,New York,New York,40.4774,-74.2591,40.9176,-73.7003,158433699,85580385068650496
1,New York,New York,40.4774,-74.2591,40.9176,-73.7003,37861099,85580392429649920
2,New York,New York,40.4774,-74.2591,40.9176,-73.7003,44111783,85580405402640384
3,New York,New York,40.4774,-74.2591,40.9176,-73.7003,208217695,85580426969755648
4,New York,New York,40.4774,-74.2591,40.9176,-73.7003,317394754,85580431545733120


In [5]:
!cat ../Mahmud_ICWSM_2012/README.txt

Top US Cities Users/Tweets Dataset - 2011

Clemens Drews, Jalal Mahmud, Jeffrey Nichols
IBM Research - Almaden

Collected for the following paper (use this as a reference for the data set):

Jalal Mahmud, Jeffrey Nichols, Clemens Drews. "Where is This Tweet From? Inferring Home Locations of Twitter Users," in Proceedings of ICWSM 2012. Dublin, Ireland.


This data set identifies 100 twitter users who tweeted at least once with a geo-tag in one of the top 100 cities in the United States by population as of April 1, 2010.  This list can be found on Wikipedia here (permalink):

http://en.wikipedia.org/w/index.php?title=List_of_United_States_cities_by_population&oldid=492433218

These users were collected through the Twitter streaming API using a track stream filtered on the geographical bounding boxes of each of the 100 cities.  The bounding boxes for each city were determined via the Google reverse geocoding service.  Tweets were collected in each location until 100 unique users were fou

In [6]:
profile_tweets = pd.read_csv("../Mahmud_ICWSM_2012/profiletweets.csv")

In [7]:
profile_tweets.head()

,USER ID,TWEET IDS
0,541,"111957685968191488,111865225178918913,11161760..."
1,607,"86238898153472000,86229062275710976,8620099568..."
2,683,"111663910209323008,111623506101420032,11157054..."
3,874,"86010769786937344,85982589193687041,8595063499..."
4,949,"111596550773424128,111511346104905728,11151076..."


In [8]:
profile_tweets.shape

(18948, 2)

In [9]:
users_and_tweets.shape

(19390, 8)

In [10]:
def convert_to_list(tweets_ids):
    return tweets_ids.split(',')

profile_tweets["TWEET IDS"] = profile_tweets["TWEET IDS"].apply(convert_to_list)

In [11]:
profile_tweets_unwind = pd.DataFrame([[x] + [z] for x, y in profile_tweets.values for z in y],columns=profile_tweets.columns)

In [12]:
profile_tweets_unwind.shape

(1024648, 2)

In [13]:
users_and_tweets.sort_values("USER ID").head()

,City,State,SW LAT,SW LONG,NE LAT,NE LONG,USER ID,TWEET ID
10677,Portland,Oregon,45.4324,-122.8370,45.6544,-122.4720,541,106850892841623555
5372,San Francisco,California,37.7034,-122.5270,37.8120,-122.3480,607,85705274593263616
6346,Columbus,Ohio,39.8086,-83.2103,40.1573,-82.7714,683,92330942122831872
5398,San Francisco,California,37.7034,-122.5270,37.8120,-122.3480,874,85710671857205248
4649,San Jose,California,37.1245,-122.0460,37.4695,-121.5890,949,82873189121789953


In [14]:
excluded_users = [x for x in users_and_tweets["USER ID"].values if x not in profile_tweets["USER ID"].values]

In [15]:
len(excluded_users)

443

In [16]:
myclient = MongoClient("mongodb://localhost:27018/")

In [17]:
db_paper = myclient["twitter_paper"]
col_paper = db_paper["tweets"]

In [18]:
access_token = "AAAAAAAAAAAAAAAAAAAAAF0sHQEAAAAA1jPUcZAGMJqv7IncKrgTRvbQ3O8%3DVylQmZJODsUTqgxnni1gLg4zz4CeLqgOL3W0O86zJpr6ADbOUM"

In [19]:
base_url = 'https://api.twitter.com/1.1/statuses/show.json?id='

Example request

In [203]:
url     = base_url + '85705274593263616'
#payload = { 
#    'tweet.fields': 'attachments,author_id,context_annotations,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,referenced_tweets,source,public_metrics,text,withheld',
#    'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type',
#    'expansions': 'attachments.poll_ids,attachments.media_keys,author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id',
#    'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'
#}
headers= {
    'Authorization': 'Bearer {}'.format(access_token) 
}
res = requests.get(url, headers=headers)

In [206]:
if res.status_code != 200:
    print("Invalid request: {}".format(res))
else:
    response_json = res.json()
    response_json["_id"] = response_json["id"]
    del response_json["id"] 
    pprint.PrettyPrinter(indent=4).pprint(response_json)

{   '_id': 85705274593263616,
    'contributors': None,
    'coordinates': {   'coordinates': [-122.40850039, 37.77666272],
                       'type': 'Point'},
    'created_at': 'Tue Jun 28 13:45:04 +0000 2011',
    'entities': {   'hashtags': [],
                    'symbols': [],
                    'urls': [],
                    'user_mentions': [   {   'id': 418,
                                             'id_str': '418',
                                             'indices': [0, 5],
                                             'name': 'Dennis Crowley',
                                             'screen_name': 'dens'}]},
    'favorite_count': 0,
    'favorited': False,
    'geo': {'coordinates': [37.77666272, -122.40850039], 'type': 'Point'},
    'id_str': '85705274593263616',
    'in_reply_to_screen_name': 'dens',
    'in_reply_to_status_id': 85672377966530560,
    'in_reply_to_status_id_str': '85672377966530560',
    'in_reply_to_user_id': 418,
    'in_reply_to_user_id

In [212]:
try:
    col_paper.insert_one(response_json)
except:
    print("Dup error")

Dup error


In [219]:
col_paper.find_one()

In [218]:
col_paper.delete_one({"_id": 85705274593263616});

Retrieve all tweets

In [20]:
headers= {
    'Authorization': 'Bearer {}'.format(access_token) 
}

In [21]:
processed_tweets = pd.DataFrame(columns={"tweet_id", "status"})

In [ ]:
TIME_SLEEP = 16 #In minutes
REQUEST_PER_SLEEP = 880

requests_done = 0
for tweet_id in profile_tweets_unwind['TWEET IDS'].values:
    url = base_url + str(tweet_id)
    res = requests.get(url, headers=headers)
    
    requests_done += 1
    
    if requests_done % REQUEST_PER_SLEEP == 0:    
        #Sleep for 15m every ~900 requests
        start_sleep = time.time()
        print("Total requests done: {} Time to sleep".format(requests_done))
        time.sleep(TIME_SLEEP * 60)
        print("Slept: {}".format(time.time() - start_sleep))    
    
    if res.status_code != 200:
        print("Invalid request: {} - {}".format(res.text, res.status_code))
        processed_tweets.loc[requests_done] = [tweet_id, 1]
        processed_tweets.to_csv("../results/tmp/processed_tweets_paper.csv", index=False)
        continue
        
    response_json = res.json()
    response_json["_id"] = response_json["id"]
    del response_json["id"] 
    
    try:
        col_paper.insert_one(response_json)
    except:
        print("Duplicate error inserting in mongo collection")
        processed_tweets.loc[requests_done] = [tweet_id, 2]
        processed_tweets.to_csv("../results/tmp/processed_tweets_paper.csv", index=False)
        continue
        
    processed_tweets.loc[requests_done] = [tweet_id, 0]  
    processed_tweets.to_csv("../results/tmp/processed_tweets_paper.csv", index=False)

Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo

Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo

Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo

Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo

Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Duplicate error inserting in mongo collection
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not authorized to see this status."}]} - 403
Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not authorized to see this status."}]} - 403
Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not authorized to see this status."}]} - 403
Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not 

...

Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404

Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404

Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not authorized to see this status."}]} - 403
Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not authorized to see this status."}]} - 403
Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not authorized to see this status."}]} - 403
Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not authorized to see this status."}]} - 403
Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not authorized to see this status."}]} - 403
Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not authorized to see this status."}]} - 403
Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not authorized to see this status."}]} - 403
Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not authorized to see this status."}]} - 403
Invalid request: {"errors":[{"code":179,"message":"Sorry, you are not authorized to see this status."}]} - 403
I

Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404

Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404
Invalid request: {"errors":[{"code":144,"message":"No status found with that ID."}]} - 404

In [43]:
tmpData = pd.read_csv("../results/tmp/processed_tweets_paper.csv")

In [45]:
len(profile_tweets_unwind['TWEET IDS'].values)

1024648

In [48]:
tmpData['status'].value_counts()

0    131407
1     41824
2      1010
Name: status, dtype: int64

In [19]:
profile_tweets_unwind.loc[:, ["TWEET IDS"]].to_csv("../Mahmud_ICWSM_2012/tweets_ids.csv", header=False, index=False)

In [24]:
profile_tweets_unwind["TWEET IDS"][2110]

'97415691115835393'

In [25]:
profile_tweets_unwind['TWEET IDS'].replace('', np.nan, inplace=True)

In [26]:
profile_tweets_unwind.shape

(1024648, 2)

In [27]:
profile_tweets_unwind.dropna(inplace=True)

In [28]:
profile_tweets_unwind.shape

(1024495, 2)

In [29]:
profile_tweets_unwind.loc[:, ["TWEET IDS"]].to_csv("../Mahmud_ICWSM_2012/tweets_ids.csv", header=False, index=False)